<a href="https://colab.research.google.com/github/alexj2100/imersao_ia_alura/blob/main/adivinhe_o_animal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
print('g-api-key')

g-api-key


In [6]:
pip install -q -U google-generativeai

In [33]:
import pathlib
import textwrap

import google.generativeai as genai

from random import randrange

from IPython.display import display
from IPython.display import Markdown

import numpy as np
import pandas as pd



def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [8]:
# Used to securely store your API key
from google.colab import userdata

In [13]:
# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [20]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [21]:
#Listagem de documentos que serão buscados
DOCUMENT1 = {
    "animal": "cobra",
    "caracteristicas": "rasteja e come ratos"}

DOCUMENT2 = {
    "animal": "águia",
    "caracteristicas": "voa e come ratos"}

DOCUMENT3 = {
    "animal": "cão",
    "caracteristicas": "corre, pula come qualquer coisa"}

DOCUMENT4 = {
    "animal": "baleia",
    "caracteristicas": "nada e come peixes"}

DOCUMENT5 = {
    "animal": "rato",
    "caracteristicas": "anda e come queijo"}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3, DOCUMENT4, DOCUMENT5]

In [24]:
df = pd.DataFrame(documents)
df.columns = ["animal", "caracteristicas"]
df

,animal,caracteristicas
0,cobra,rasteja e come ratos
1,águia,voa e come ratos
2,cão,"corre, pula come qualquer coisa"
3,baleia,nada e come peixes
4,rato,anda e come queijo


In [25]:
model = "models/embedding-001"

In [26]:
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [27]:
df["Embeddings"] = df.apply(lambda row: embed_fn(row["animal"], row["caracteristicas"]), axis=1)
df

,animal,caracteristicas,Embeddings
0,cobra,rasteja e come ratos,"[0.02390636, 0.0006585114, -0.05542453, -0.008..."
1,águia,voa e come ratos,"[0.037311915, -0.008570076, -0.04421841, -0.02..."
2,cão,"corre, pula come qualquer coisa","[0.0128075825, -0.03206677, -0.037184715, -0.0..."
3,baleia,nada e come peixes,"[0.049467977, 0.00824817, -0.0358011, 0.000296..."
4,rato,anda e come queijo,"[0.023643695, -0.017406376, -0.059219275, -0.0..."


In [28]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["caracteristicas"]

In [29]:
#Listando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [30]:
generation_config = {
  "candidate_count": 1,
  "temperature": 0.5,
}

In [31]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [32]:
safety_settings={
    'HATE': 'BLOCK_NONE',
    'HARASSMENT': 'BLOCK_NONE',
    'SEXUAL' : 'BLOCK_NONE',
    'DANGEROUS' : 'BLOCK_NONE'
    }

In [64]:
print('Eu pensei em um animal, tente adivinhar\n')
indice = randrange(5)

prompt = input('Pergunte uma característica do animal: ')

while prompt != "fim":
  response = gerar_e_buscar_consulta(prompt, df, model)
  if df.at[indice,'caracteristicas'] == response:
    print('sim')
  else:
    print('não')
  prompt = input('Qual é o animal: ')
  if df.at[indice,'animal'] == prompt:
    print('voce acertou! Parabéns')
    prompt = "fim"
  else:
     print('errou! tente outra vez ou digite fim para encerrar')
     prompt = input('Pergunte uma característica do animal: ')

Eu pensei em um animal, tente adivinhar

Pergunte uma característica do animal: ele anda?
não
Qual é o animal: águia
errou! tente outra vez ou digite fim para encerrar
Pergunte uma característica do animal: ele come queijo?
não
Qual é o animal: cão
errou! tente outra vez ou digite fim para encerrar
Pergunte uma característica do animal: ele rasteja?
não
Qual é o animal: baleia
voce acertou! Parabéns
